# Import

In [3]:
library(GNAR)

In [4]:
library(jsonlite)

# function

In [5]:
cal_mse <- function(data, net_input, lags, missing_rate) {
      train_ratio <- floor(nrow(data) * 0.8)
      test_ratio <- ceiling(nrow(data) * 0.2)
      df <- data
      num_cols <- ncol(df)
      for (i in 1:num_cols) {
              indices_to_replace <- sample(c(1:(nrow(df) - test_ratio - lags)), 
                                           size = missing_rate * (nrow(df) - test_ratio - lags))
              df[indices_to_replace, i] <- NA
            }
      nafit <- GNARfit(vts = df[1:train_ratio,], net = net_input, alphaOrder = lags, betaOrder = rep(1, lags))
      predict<- predict(nafit,n.ahead = test_ratio)
    
      mse_each <- numeric(num_cols)
      for (i in 1:num_cols) {
          mse_each[i] <- mean((predict[,i] - df[(train_ratio+1):nrow(df),i])**2)
      }
      mse_total <- mean(mse_each)
      return(list('each'= mse_each, 'total'=mse_total))
    }

In [6]:
block_cal_mse <- function(data, net_input, lags, mindex) {
      train_ratio <- floor(nrow(data) * 0.8)
      test_ratio <- ceiling(nrow(data) * 0.2)
      df <- data
      num_cols <- ncol(df)
      for (i in 1:num_cols) {
              df[mindex[[i]],i] <- NA
            }
      nafit <- GNARfit(vts = df[1:train_ratio,], net = net_input, alphaOrder = lags, betaOrder = rep(1, lags))
      predict<- predict(nafit,n.ahead = test_ratio)
    
      mse_each <- numeric(num_cols)
      for (i in 1:num_cols) {
          mse_each[i] <- mean((predict[,i] - df[(train_ratio+1):nrow(df),i])**2)
        }
      mse_total <- mean(mse_each)
      return(list('each'= mse_each, 'total'=mse_total))
    }

# fivenodes

## random

In [1330]:
fixed_data <- fiveVTS
fixed_net_input <- fiveNet
fixed_lags <- 2

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,1.257729
0.1,1.252735
0.2,1.265524
0.3,1.250937
0.4,1.271402
0.5,1.278707
0.6,1.301504
0.7,1.301504
0.8,1.301504


In [1331]:
results_df['dataset'] = 'FiveVTS'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
FiveVTS_results <- results_df

## block

In [1345]:
FiveVTS_mindex <- fromJSON("FiveVTS_mindex.json")

In [1348]:
fixed_data <- fiveVTS
fixed_net_input <- fiveNet
fixed_lags <- 2
fixed_mindex <- FiveVTS_mindex

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.125,1.272369


In [1349]:
results_df['dataset'] = 'FiveVTS'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
FiveVTS_block_results <- results_df

# Chickenpox

## random

In [1351]:
Chickenpox<-read.csv('./data/Chickenpox.csv',header = TRUE)

In [1352]:
Chickenpox_w<-read.csv('./data/Chickenpox_w.csv',header = FALSE)[2:21,2:21]

In [1385]:
fixed_data <- as.matrix(Chickenpox[,2:ncol(Chickenpox)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Chickenpox_w))
fixed_lags <- 4

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8)

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,1.016342
0.1,1.016460
0.2,1.015845
0.3,1.018481
0.4,1.020843
0.5,1.019158
0.6,1.021600
0.7,1.021411
0.8,1.020084


In [1386]:
results_df['dataset'] = 'Chickenpox'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Chickenpox_results <- results_df

## block

In [1387]:
Chickenpox_mindex <- fromJSON("Chickenpox_mindex.json")

In [1388]:
fixed_data <- as.matrix(Chickenpox[,2:ncol(Chickenpox)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Chickenpox_w))
fixed_lags <- 4
fixed_mindex <- Chickenpox_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.2884615,1.017119


In [1389]:
results_df['dataset'] = 'Chickenpox'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Chickenpox_block_results <- results_df

# PedalMe

## random

In [1390]:
PedalMe<-read.csv('./data/PedalMe.csv',header = TRUE)

In [1391]:
PedalMe_w<-read.csv('./data/PedalMe_w.csv',header = FALSE)[2:16,2:16]

In [1455]:
fixed_data <- as.matrix(PedalMe[,2:ncol(PedalMe)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(PedalMe_w))
fixed_lags <- 4

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6,0.7, 0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,0.5016028
0.1,0.4971018
0.2,0.5027820
0.3,0.5148579
0.4,0.5226205
0.5,0.5917800
0.6,0.4917371
0.7,0.4917371
0.8,0.4917371


In [1456]:
results_df['dataset'] = 'Pedalme'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Pedalme_results <- results_df

## block

In [1457]:
Pedalme_mindex <- fromJSON("Pedalme_mindex.json")

In [1458]:
fixed_data <- as.matrix(PedalMe[,2:ncol(PedalMe)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(PedalMe_w))
fixed_lags <- 4
fixed_mindex <- Pedalme_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.2941176,0.4916068


In [1459]:
results_df['dataset'] = 'Pedalme'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Pedalme_block_results <- results_df

# WikiMath

## random

In [17]:
WikiMath <- read.csv('./data/WikiMath.csv',header = TRUE)

In [18]:
WikiMath_w<-read.csv('./data/WikiMath_w.csv',header = FALSE)[2:1069,2:1069]

In [1552]:
fixed_data <- as.matrix(WikiMath[,2:ncol(WikiMath)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(WikiMath_w))
fixed_lags <- 8

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,0.9022260
0.1,0.9015769
0.2,0.9015677
0.3,0.9112567
0.4,0.9576448
0.5,0.9042646
0.6,0.9465203
0.7,0.9607108
0.8,2.0823862


In [1554]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.8)

$each
   [1] 0.71660920 0.33609588 1.53037835 0.57188075 0.67904028 0.86770945
   [7] 0.94211227 1.23095353 0.25556277 1.41021789 1.11902155 0.12077769
  [13] 0.35168692 1.03547248 0.38763710 1.67286281 0.64568848 1.02399009
  [19] 0.47051511 1.39830665 0.52747481 0.87359633 3.91639405 0.80974871
  [25] 1.52352970 0.95480997 0.59427156 1.07384103 1.16050100 0.81237884
  [31] 0.08436705 0.58288641 0.61094112 1.77501611 0.83218848 0.28554788
  [37] 0.85679380 0.78617997 1.08285615 1.27673759 1.12110059 0.64807166
  [43] 0.67397983 0.71271550 1.87262236 0.12420030 0.55595005 0.43516665
  [49] 0.76961792 0.88195964 0.40965569 0.25882114 0.65436935 1.04090279
  [55] 1.10618126 0.58081812 0.90164899 3.86663464 0.53033716 0.97211000
  [61] 0.31678610 0.55785350 0.60934602 0.95718573 0.85895199 2.05441525
  [67] 0.60814889 0.43058117 1.01848970 0.98091231 0.14800767 1.01409833
  [73] 1.05590401 0.52674403 1.32274716 0.80007640 0.78041933 0.83791991
  [79] 0.51026914 0.38042566 1.33540742 1.00353609 0.63474628 0.75538373
  [85] 1.08267748 0.72135313 2.29833341 0.84068786 0.53197956 0.85959737
  [91] 0.14091156 0.77591802 0.93412760 4.81659441 0.84927060 0.89518463
  [97] 0.51374776 2.93268640 0.68960052 1.19716633 0.76470626 0.86897399
 [103] 0.04780388 0.88209600 0.32334916 0.57178842 0.65880942 0.75211820
 [109] 0.87537028 1.47330628 0.60276423 0.23344938 0.78051107 0.81103356
 [115] 0.67450281 0.70352706 1.65561277 0.44569400 0.57628057 4.86089961
 [121] 2.91953626 0.48247269 0.75937769 0.53268481 1.43877390 2.16968798
 [127] 0.98454587 0.91300652 1.06506829 0.85582071 4.71409578 1.00157750
 [133] 0.10163096 1.24809671 1.40404140 1.43001425 1.26345838 1.38900945
 [139] 1.07859369 1.17680003 1.37956355 0.91941577 2.16635302 0.92464786
 [145] 0.95160413 0.96189061 1.02526391 0.20885687 1.32637413 0.57811094
 [151] 0.39081539 0.99082808 0.65009011 0.94561368 0.22625778 0.48141959
 [157] 0.85041312 0.72740764 0.73194285 1.92466970 0.38248675 0.76315234
 [163] 0.07185972 0.68984030 0.72526191 2.19828458 0.71303222 1.06066010
 [169] 0.95012218 0.83251381 1.06902082 1.07562044 1.10584501 1.22421445
 [175] 1.13842887 1.13973300 0.23033995 1.11115013 0.46195868 1.10830054
 [181] 1.29817313 1.48189062 0.80751857 1.21626446 0.74115313 0.20312591
 [187] 0.17644196 1.57100645 0.96643706 1.14319556 0.42683159 0.06475266
 [193] 0.79927046 0.59910421 0.45828446 0.76018816 1.25520996 0.58344896
 [199] 0.81306238 0.75428490 1.02752745 0.95402040 1.07577348 0.69238312
 [205] 1.07490766 0.94025223 0.60875796 2.07292587 0.96617355 0.74130278
 [211] 0.74626069 0.04431899 0.68837094 0.68684162 0.66449914 1.80682087
 [217] 0.09436692 0.34503255 1.67267173 0.99217006 0.35176753 0.72894682
 [223] 1.27555669 2.26718395 0.91914654 1.18042992 2.39825568 0.64465290
 [229] 0.58248515 0.73909313 0.85664360 0.71949733 0.46887161 0.83911827
 [235] 1.14995480 1.02933577 1.30662255 0.87519448 0.04188833 1.57082318
 [241] 0.06511363 0.92167752 1.42668962 1.05435031 1.46879326 0.52149085
 [247] 1.10695973 1.28927484 0.94865291 0.89184928 0.36456450 1.83915549
 [253] 0.84010170 0.57769680 0.79776814 0.78235034 0.90935727 1.41900398
 [259] 2.15925793 0.54376125 0.99547718 0.37733158 1.32702543 0.98634436
 [265] 1.06498797 1.65905978 0.89374655 1.15663272 0.85188898 0.81171327
 [271] 1.35073410 1.19473419 1.13771282 1.30413601 1.20715723 0.97055940
 [277] 0.47547999 0.84535422 0.65495293 1.10310720 1.24458576 0.76278818
 [283] 1.23596702 0.89281235 0.47991338 0.45096342 1.37678493 1.16905232
 [289] 0.84491799 4.26667736 0.48970845 0.92410680 0.68729775 1.05094219
 [295] 1.22006335 2.05566951 0.89729722 0.99384029 0.59127811 0.87022444
 [301] 0.76514600 0.79335960 0.51853568 1.24601602 1.61590579 1.10996958
 [307] 1.02263436 0.95806066 1.94661944 1.00303986 0.68701466 0.93257548
 [313] 0.67049992 0.55748482 0.96042899 0.82428931 0.87205426 0.68337896
 [319] 0.41161265 1.47370270 0.88393635 1.22760584 0.96709517 1.49210331
 [325] 0.65376335 1.06110942 0.87364728 0.13132670 0

In [ ]:
results_df['dataset'] = 'Wikimath'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Wikimath_results <- results_df

## block

In [19]:
Wikimath_mindex <- fromJSON("./GNAR_weight_matrix/Wikimath_mindex.json")

In [20]:
fixed_data <- as.matrix(WikiMath[,2:ncol(WikiMath)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(WikiMath_w))
fixed_lags <- 8
fixed_mindex <- Wikimath_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.6002771,0.9083379


In [21]:
results_df['dataset'] = 'Wikimath'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Wikimath_block_results <- results_df

# Windmillsmall

## random

In [9]:
Windmillsmall<-read.csv('./data/Windmillsmall.csv',header = TRUE)

In [10]:
Windmillsmall_w<-read.csv('./data/Windmillsmall_w.csv',header = FALSE)[2:12,2:12]

계속 오류남;;

In [885]:
# fixed_data <- as.matrix(Windmillsmall[,2:ncol(Windmillsmall)])
# fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Windmillsmall_w))
# fixed_lags <- 8

# results_df <- data.frame(mrate = numeric(),
#                          mse = numeric())

# missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8) 

# for (rate in missing_rates) {
#   result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
#   results_df <- rbind(results_df, data.frame(mrate = rate, mse = result))
# }
# results_df

In [1543]:
results_df <- data.frame(
  mrate = c(0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0),
  mse = c(0.980465007853614,0.980465007853614,0.980742615896619,0.980465007853614, 0.980529157673243, 0.980548927794843,0.980486712335988,0.980471934178215, 0.980486581012802)
)

In [1544]:
results_df['dataset'] = 'Windmillsmall'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Windmillsmall_results <- results_df

In [1505]:
fixed_data <- as.matrix(Windmillsmall[,2:ncol(Windmillsmall)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Windmillsmall_w))
fixed_lags <- 8

In [1524]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.8)[2]

$total
[1] 0.980465

In [1541]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.7)[2]

$total
[1] 0.980465

In [1527]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.6)[2]

$total
[1] 0.9807426

In [1512]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.5)[2]

$total
[1] 0.980465

In [1511]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.4)[2]

$total
[1] 0.9805292

In [1510]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.3)[2]

$total
[1] 0.9805489

In [1509]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.2)[2]

$total
[1] 0.9804867

In [1508]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.1)[2]

$total
[1] 0.9804719

In [1507]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0)[2]

$total
[1] 0.9804866

## block

In [7]:
Windmillsmall_mindex <- fromJSON("Windmillsmall_mindex.json")

In [11]:
fixed_data <- as.matrix(Windmillsmall[,2:ncol(Windmillsmall)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Windmillsmall_w))
fixed_lags <- 8
fixed_mindex <- Windmillsmall_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.2861885,0.9804698


In [12]:
results_df['dataset'] = 'Windmillsmall'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Windmillsmall_block_results <- results_df

# MontevideoBus

## random

In [877]:
MontevideoBus<-read.csv('./data/MontevideoBus.csv',header = TRUE)

In [878]:
MontevideoBus_w<-read.csv('./data/MontevideoBus_w.csv',header = FALSE)[2:676,2:676]

In [1498]:
fixed_data <- as.matrix(MontevideoBus[,2:ncol(MontevideoBus)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(MontevideoBus_w))
fixed_lags <- 8

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,1.002868
0.1,1.002744
0.2,1.002750
0.3,1.002771
0.4,1.001533
0.5,1.002278
0.6,1.002135
0.7,1.003126
0.8,1.003196


In [1499]:
results_df['dataset'] = 'MontevideoBus'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
MontevideoBus_results <- results_df

## block

In [1226]:
MontevideoBus_mindex <- fromJSON("MontevideoBus_mindex.json")

In [1501]:
fixed_data <- as.matrix(MontevideoBus[,2:ncol(MontevideoBus)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(MontevideoBus_w))
fixed_lags <- 8
fixed_mindex <- MontevideoBus_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.1495439,1.002894


In [1502]:
results_df['dataset'] = 'MontevideoBus'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
MontevideoBus_block_results <- results_df

# Final Data

In [1563]:
rbind(FiveVTS_results,Chickenpox_results,Pedalme_results,Wikimath_results,Windmillsmall_results,MontevideoBus_results)

mrate,mse,dataset,method,mtype,lags,nof_filters,inter_method,epoch,calculation_time,model
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
0.0,1.2577286,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.1,1.2527354,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.2,1.2655242,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.3,1.2509369,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.4,1.2714016,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.5,1.2787071,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.6,1.3015036,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.7,1.3015036,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.8,1.3015036,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR


In [1564]:
rbind(FiveVTS_block_results,Chickenpox_block_results,Pedalme_block_results,Wikimath_block_results,Windmillsmall_block_results,MontevideoBus_block_results)

mrate,mse,dataset,method,mtype,lags,nof_filters,inter_method,epoch,calculation_time,model
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
0.12500000,1.2723690,FiveVTS,GNAR,block,2,NA,NA,NA,NA,GNAR
0.28846154,1.0171190,Chickenpox,GNAR,block,4,NA,NA,NA,NA,GNAR
0.29411765,0.4916068,Pedalme,GNAR,block,4,NA,NA,NA,NA,GNAR
0.12024781,0.9009986,Wikimath,GNAR,block,8,NA,NA,NA,NA,GNAR
0.03252143,0.9804863,Windmillsmall,GNAR,block,8,NA,NA,NA,NA,GNAR
0.14954389,1.0028939,MontevideoBus,GNAR,block,8,NA,NA,NA,NA,GNAR


In [1565]:
final_df <- rbind(FiveVTS_results,Chickenpox_results,Pedalme_results,Wikimath_results,Windmillsmall_results,MontevideoBus_results,
     FiveVTS_block_results,Chickenpox_block_results,Pedalme_block_results,Wikimath_block_results,Windmillsmall_block_results,MontevideoBus_block_results)

In [1566]:
write.csv(final_df, file = "./R_GNAR_results.csv", row.names = FALSE)